### Мемоизация — это техника оптимизации, при которой результаты выполнения функции кешируются, чтобы избежать повторных вычислений.

Источник - https://habr.com/ru/articles/335866/

In [1]:
import time
def clock(func):
    def clocked(*args, **kwargs):
        t0 = time.time()

        result = func(*args, **kwargs)  # вызов декорированной функции

        elapsed = time.time() - t0
        name = func.__name__
        arg_1st = []
        if args:
            arg_1st.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_1st.append(', '.join(pairs))
        arg_str = ', '.join(arg_1st)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [2]:
def memoize(f):  
    # Определение декоратора memoize, который принимает функцию f в качестве аргумента.
    cache = {}  
    # Локальный словарь cache используется для хранения уже вычисленных результатов функции.
    
    def decorate(*args):  
        # Внутренняя функция decorate принимает произвольное количество аргументов (*args).
        if args in cache:  
            # Проверяет, есть ли эти аргументы в словаре cache.
            return cache[args]  
            # Если есть, возвращает сохраненное значение, избегая повторного выполнения функции.
        else:  
            cache[args] = f(*args)  
            # Если нет, вычисляет результат выполнения функции f с этими аргументами,
            # сохраняет результат в словаре cache по ключу args.
            return cache[args]  
            # Возвращает вычисленный результат.

    return decorate  
    # Возвращает функцию decorate, которая заменяет исходную функцию f.

In [3]:
# Базовый пример - функция подсчета числа Фибоначчи
@memoize
def fibonacci_memo(n):
    if n < 2:
        return n
    return fibonacci_memo(n - 1) + fibonacci_memo(n - 2)

In [4]:
def fibonacci_non_memo(n):
    if n < 2:
        return n  
    return fibonacci_non_memo(n - 1) + fibonacci_non_memo(n - 2)

In [5]:
@clock
def many_heavy_comp(func, n):
    for i in range(n):
        func(i)
    print(func(n))

In [6]:
many_heavy_comp(fibonacci_memo, 100)

354224848179261915075
[0.00022697s] many_heavy_comp(<function memoize.<locals>.decorate at 0x7f698c0f2480>, 100) -> None


In [7]:
many_heavy_comp(fibonacci_non_memo, 20)

6765
[0.00172019s] many_heavy_comp(<function fibonacci_non_memo at 0x7f698c0f2520>, 20) -> None


Без мемоизации даже 20 вычислений Фибоначчи медленнее, чем вычисление 100 Фибоначчи с мемоизации

### Кеширование - более общее понятие чем мемоизация. В кеше можно сохранять любые данные для их переиспользования, а при мемоизации сохраняются только результаты выполненных в процессе работы функций, т.е. мемоизация - частный случай кеширования

In [8]:
from functools import lru_cache

In [9]:
@lru_cache(maxsize=128)  # Декоратор lru_cache кэширует до 128 последних результатов выполнения функции
def heavy_computation(x):
    time.sleep(2)  # Имитация долгих вычислений (2 секунды)
    return x ** x

In [10]:
def heavy_computation_not_cached(x):
    time.sleep(2)  # Имитация долгих вычислений (2 секунды)
    return x ** x

In [11]:
many_heavy_comp(heavy_computation, 10)

10000000000
[22.00261331s] many_heavy_comp(<functools._lru_cache_wrapper object at 0x7f698c41fab0>, 10) -> None


In [12]:
many_heavy_comp(heavy_computation, 10)

10000000000
[0.00004673s] many_heavy_comp(<functools._lru_cache_wrapper object at 0x7f698c41fab0>, 10) -> None


In [13]:
many_heavy_comp(heavy_computation_not_cached, 10)

10000000000
[22.00169039s] many_heavy_comp(<function heavy_computation_not_cached at 0x7f698c0f1e40>, 10) -> None


In [14]:
many_heavy_comp(heavy_computation_not_cached, 10)

10000000000
[22.00229716s] many_heavy_comp(<function heavy_computation_not_cached at 0x7f698c0f1e40>, 10) -> None


Как мы видим - без кеширования функция при новом выводе выполняется с нуля и занимает 22 секунды. С ним - меньше 0.0001s

Другие примеры кеширования с помощью functools:

In [22]:
from functools import cache, cached_property

@cache
def factorial_cache(n):
    return 1 if n == 0 else n * factorial_cache(n - 1)
    

In [19]:
def factorial_non_cache(n):
    return 1 if n == 0 else n * factorial_non_cache(n - 1)

In [20]:
many_heavy_comp(factorial_cache, 20)

2432902008176640000
[0.00004625s] many_heavy_comp(<functools._lru_cache_wrapper object at 0x7f698c178ca0>, 20) -> None


In [21]:
many_heavy_comp(factorial_non_cache, 20)

2432902008176640000
[0.00005007s] many_heavy_comp(<function factorial_non_cache at 0x7f696f73ca40>, 20) -> None


In [26]:
class Circle:
    def __init__(self, radius):
        self.radius = radius

    
    @cached_property
    @clock
    def area(self):
        print("Вычисление площади...")
        return 3.14159 * self.radius ** 2

circle = Circle(5)

In [27]:
circle.area

Вычисление площади...
[0.00004268s] area(<__main__.Circle object at 0x7f696f732cf0>) -> 78.53975


78.53975

In [29]:
circle.area

78.53975

cached_propery применяется для объектов класса, cache - кеширует все результаты вызова функций, lru_cache - только n (используется для ограничения сохраненных результатов)

### Интенирование строк - одинаковые строки могут быть хранимы в памяти как один объект, что экономит ресурсы.

In [15]:
a = "hello"
b = "hello"

In [16]:
a is b

True